<h1 style='text-align:center'>RECONOCIMIENTO</h1>

 toma de imagenes parte1


In [2]:
import cv2
import os

nombre = "Nombre_persona"  
ruta = f'dataset/{nombre}'
os.makedirs(ruta, exist_ok=True)

cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        rostro = gray[y:y+h, x:x+w]
        cv2.imwrite(f"{ruta}/{count}.jpg", rostro)
        count += 1
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

    cv2.imshow('Captura de rostros', frame)
    if cv2.waitKey(1) == 27 or count >= 20:
        break

cap.release()
cv2.destroyAllWindows()


In [3]:
from cv2 import face
import numpy as np

def get_images_and_labels(dataset_path='dataset'):
    faces = []
    labels = []
    label_map = {}
    label_id = 0

    for person in os.listdir(dataset_path):
        person_path = os.path.join(dataset_path, person)
        if not os.path.isdir(person_path):
            continue
        label_map[label_id] = person
        for image_name in os.listdir(person_path):
            img_path = os.path.join(person_path, image_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            faces.append(img)
            labels.append(label_id)
        label_id += 1

    return faces, np.array(labels), label_map

faces, labels, label_map = get_images_and_labels()
recognizer = face.LBPHFaceRecognizer_create()
recognizer.train(faces, labels)

os.makedirs("recognizer", exist_ok=True)
recognizer.save('recognizer/trainer.yml')

import pickle
with open("recognizer/labels.pickle", "wb") as f:
    pickle.dump(label_map,f)

In [4]:
import pickle

recognizer = face.LBPHFaceRecognizer_create()
recognizer.read('recognizer/trainer.yml')

with open("recognizer/labels.pickle", "rb") as f:
    label_map = pickle.load(f)

cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        rostro = gray[y:y+h, x:x+w]
        id_, conf = recognizer.predict(rostro)
        if conf < 70:
            nombre = label_map[id_]
        else:
            nombre = "Desconocido"

        cv2.putText(frame, nombre, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    cv2.imshow('Reconocimiento de rostros', frame)
    if cv2.waitKey(1) == 27:
        break

cap.release()
cv2.destroyAllWindows()